## Arundine

* Arundine, or 3,3-Diindolylmethane, is a natural compound formed in the stomach from its precursor Indol-3-carbinol in Brassicacea [Diindolylmethane (DIM) Information Resource Center, U Cali Berkeley](https://www.diindolylmethane-dim.com/), [Biersack 2020](https://www.cdrjournal.com/article/view/3692)
* neuroprotective action and BBB-permeability:
    * the MPTPp in-vivo mouse model of Parkinson's disease (para-phenyl–substituted diindolylmethane (C-DIM), [De Miranda et al 2013](https://jpet.aspetjournals.org/content/345/1/125), [De Miranda et al 2015](https://academic.oup.com/toxsci/article/143/2/360/1676838))
    * oxidative stress induced apoptosis in hippocampal neuronal cells ([Lee et al 2020](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7023184/))
    * primary hippocampal cell cultures exposed to ischemia ([Rzemieniec et al 2019](https://link.springer.com/article/10.1007/s10495-019-01522-2))
* antiinflammatory in fibroblasts from rheumatoid arthritis patients ([Du et al 2019](https://www.frontiersin.org/articles/10.3389/fimmu.2019.01620/full))
* targets in our drug-target network
    * [RGS4](https://www.genecards.org/cgi-bin/carddisp.pl?gene=RGS4): Regulator Of G Protein Signaling 4. Inhibits signal transduction by increasing the GTPase activity of G protein alpha subunits thereby driving them into their inactive GDP-bound form.
    * [GPR84](https://www.genecards.org/cgi-bin/carddisp.pl?gene=GPR84): G-Protein Coupled Receptor 84.  Receptor for medium-chain free fatty acid (FFA) with carbon chain lengths of C9 to C14. Capric acid (C10:0), undecanoic acid (C11:0) and lauric acid (C12:0) are the most potent agonists. Not activated by short-chain and long-chain saturated and unsaturated FFAs. Activation by medium-chain free fatty acid is coupled to a pertussis toxin sensitive G(i/o) protein pathway. May have important roles in processes from fatty acid metabolism to regulation of the immune system.
* other targets
    * [AHR](https://www.genecards.org/cgi-bin/carddisp.pl?gene=AHR): aryl hydrocarbon receptor: selective aryl hydrocarbon receptor modulator.  Ligand-activated transcription factor that enables cells to adapt to changing conditions by sensing compounds from the environment, diet, microbiome and cellular metabolism, and which plays important roles in development, immunity and cancer (PubMed:30373764, 23275542, 7961644, 32818467). ChEMBL evidence from mouse (Kd 90 nM) and human (missing activity data) studies
* multiple suggested molecular mechanisms relevant to Alzheimer's disease
    * AHR and histone deacylase dependent apoptosis and autophagy (neuroprotective effect [Rzemieniec et al 2019](https://link.springer.com/article/10.1007/s10495-019-01522-2))
    * activates TrkB/Akt signal pathway and thus promotes BDNF and antioxidant enzyme formation (neuroprotective effect [Lee et al 2020](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7023184/))
    * inhibits MAPK and AKT/mTOR pathway an prevents TNF-alfa induced proinflammatory factors ([Du et al 2019](https://www.frontiersin.org/articles/10.3389/fimmu.2019.01620/full))
* clinical trials ([ChEMBL](https://www.ebi.ac.uk/chembl/compound_report_card/CHEMBL446452/))
    * cervical cancer (phase 3)
    * breast cancer (phase 2)
    * prostate adenocarcinoma (phase 1)
* other indications
    * gastrointestinal cancer ([Kim 2016](https://www.mdpi.com/1422-0067/17/7/1155))

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from toolbox import wrappers
from toolbox import network_utilities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import attila_utils
import repos_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


Gene-gene network

In [2]:
network_cheng = wrappers.get_network('../../resources/PPI/Cheng2019/network.sif', only_lcc = True)
ggn = network_cheng # gene-gene network

ID converters from Entrez

In [3]:
fpath = '../../resources/hgnc/hgnc_complete_set.txt'
hgnc = pd.read_csv(fpath, sep='\t', usecols=['uniprot_ids', 'symbol', 'entrez_id'], dtype={'entrez_id': str}).dropna()
uniprot2ids = hgnc.set_index('uniprot_ids')
symbol2ids = hgnc.set_index('symbol')
entrez2ids = hgnc.set_index('entrez_id')

Gene sets

In [4]:
def convert_filter(l, x2ids=symbol2ids, remove_not_in_ggn=True):
    val = [x2ids.loc[u, 'entrez_id'] for u in l if (not remove_not_in_ggn) or (x2ids.loc[u, 'entrez_id'] in ggn.nodes)]
    return(val)


In [5]:
fpath = '../../results/2021-12-02-proximity-various-ADgenesets/merged-proximity-chembl.xlsx'
gsets = dict()
gsets['knowledge'] = convert_filter(pd.read_excel(fpath, sheet_name='knowledge', header=None).iloc[:, 0].to_list(), remove_not_in_ggn=True)

Selected top drugs from network proximity based screen

In [6]:
sel_drugs = {'Arundine': 'CHEMBL446452',
            'Chenodiol': 'CHEMBL240597',
            'Cysteamine': 'CHEMBL602',
            'Cysteamine hydrochloride': 'CHEMBL1256137'}

Drug-target network.  Note that Q9NQS5/GPR84, a target of Arundine, is not in the gene-gene network.

In [7]:
fpath = '../../results/2021-10-24-chembl-query/chembl-avg_p_activities-phase_3_4-avg_p_act_5plus.csv'
dtn = pd.read_csv(fpath).groupby('drug_chembl_id').apply(lambda x: x.target_uniprot_ac.to_list())
dtn_sel = dtn.loc[sel_drugs.values()].to_frame('uniprot_ac')
dtn_sel = pd.concat([dtn_sel.index.to_frame(), dtn_sel], axis=1)
dtn_sel = dtn_sel.rename({v: k for k, v in sel_drugs.items()}, axis=0)

def add_id(ID='symbol', remove_not_in_ggn=False):
#Translate uniprot_ac to ID type; possibly remove gene if not in gene-gene network
    def helper(l):
        val = [uniprot2ids.loc[u, ID] for u in l if (not remove_not_in_ggn) or (uniprot2ids.loc[u, 'entrez_id'] in ggn.nodes)]
        return(val)
    return(dtn_sel.uniprot_ac.apply(helper))

dtn_sel['symbol'] = add_id('symbol', remove_not_in_ggn=False)
dtn_sel['entrez_id'] = add_id('entrez_id', remove_not_in_ggn=False)
dtn_sel['symbol_ggn'] = add_id('symbol', remove_not_in_ggn=True)
dtn_sel['entrez_id_ggn'] = add_id('entrez_id', remove_not_in_ggn=True)
dtn_sel

,drug_chembl_id,uniprot_ac,symbol,entrez_id,symbol_ggn,entrez_id_ggn
drug_chembl_id,,,,,,
Arundine,CHEMBL446452,"[P49798, Q9NQS5]","[RGS4, GPR84]","[5999, 53831]",[RGS4],[5999]
Chenodiol,CHEMBL240597,"[P49798, Q12908]","[RGS4, SLC10A2]","[5999, 6555]","[RGS4, SLC10A2]","[5999, 6555]"
Cysteamine,CHEMBL602,"[P08684, P49798]","[CYP3A4, RGS4]","[1576, 5999]","[CYP3A4, RGS4]","[1576, 5999]"
Cysteamine hydrochloride,CHEMBL1256137,"[O75496, P02545, P53350, P54132, Q16236, Q9NUW8]","[GMNN, LMNA, PLK1, BLM, NFE2L2, TDP1]","[51053, 4000, 5347, 641, 4780, 55775]","[GMNN, LMNA, PLK1, BLM, NFE2L2, TDP1]","[51053, 4000, 5347, 641, 4780, 55775]"


## Results

In [8]:
{source_id: network_utilities.get_shortest_path_length_between(ggn, source_id, target_id) for source_id in gsets['knowledge']}

NameError: name 'target_id' is not defined

In [ ]:
%connect_info